In [1]:
%load_ext autoreload
%autoreload 2

from util_0803_3 import *
from myalgorithm_0803_2 import algorithm

import pandas as pd

## 실험 내용
- 라이더별로 이미 최적을 확인하면 바로 리턴하기

In [26]:
problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_15.json'
# problem_file = '../alg_test_problems_20240429/TEST_K200_1.json'

testcase_name = problem_file.split('\\')[-1].split('.')[0]

start_time = time.time()

## ------------------- 기본 변수 할당 ---------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

timelimit = 60

## ------------------- 초기 상태 할당 코드 -------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

all_bundles = []
for ord in ALL_ORDERS:
    new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
    car_rider.available_number -= 1
    all_bundles.append(new_bundle)

## ------------------  주문 조합 가능성 행렬 제작하기 -----------------------------

order_comb_possibility = [[True] * K for _ in range(K)]

for i in range(len(all_bundles)):
    for j in range(i + 1, len(all_bundles)):
        bundle1 = all_bundles[i]
        bundle2 = all_bundles[j]

        order_num1 = bundle1.shop_seq[0]
        order_num2 = bundle2.shop_seq[0]

        ip = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, bundle1, bundle2)

        if not ip:
            order_comb_possibility[order_num1][order_num2] = False
            order_comb_possibility[order_num2][order_num1] = False

optimized_order_perms = [dict(), dict(), dict()] # optimized_order_perms[rider_i] = {orders_sorted: 최적 번들}

min_init_cost = inf
min_init_cost_bundles = []
min_init_cost_rider_availables = []

weight_grid = [(1, -1), (1, -1.5), (1, -2), (1, -2.5), (1, -3), (1, -3.5), (0.5, 1), (0.5, 2), (0.5, 3)]

temp_process_time = -1
for weight1, weight2 in weight_grid:
    cur_time = time.time()

    if cur_time - start_time > 30:
        break

    temp_start_time = time.time()

    bundles, result_rider_availables, cost = get_init_bundle_4_order_bundle_prefered_with_reassigning_riders(
            K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, order_comb_possibility, optimized_order_perms)
    temp_end_time = time.time()

    temp_process_time = temp_end_time - temp_start_time

    print(temp_process_time, cost)

    if cost < min_init_cost:
        min_init_cost = cost
        min_init_cost_bundles = bundles
        min_init_cost_rider_availables = result_rider_availables

0.338794469833374 4340.945499999999
0.11422896385192871 4345.277499999999
0.05099940299987793 4355.6775
0.0805656909942627 4361.633999999999
0.046602725982666016 4383.185999999999
0.03699994087219238 4410.124499999999
0.1769266128540039 4531.972999999999
0.14957594871520996 4552.808999999999
0.15416669845581055 4640.377


In [27]:
[len(v) for v in optimized_order_perms]

[833, 833, 833]

In [ ]:
# 기본
0.34907984733581543 4412.215499999999
0.3235352039337158 4346.513999999999
0.33890724182128906 4390.8944999999985
0.3010122776031494 4406.860499999999
0.2786216735839844 4454.075499999999
0.3336360454559326 4468.93
0.355849027633667 4559.7435
0.3375818729400635 4629.9145
0.3975405693054199 4618.075999999999

# 불가능한 셋 확인
0.3295269012451172 4412.215499999999
0.21972107887268066 4337.919499999999
0.13394522666931152 4370.759999999999
0.1391456127166748 4389.153499999999
0.09472036361694336 4457.306
0.1281287670135498 4476.911
0.298184871673584 4551.579999999999
0.2136082649230957 4633.433999999999
0.19490742683410645 4731.247

# reduced 불가능한 셋 확인
0.3341975212097168 4412.215499999999
0.2240755558013916 4337.919499999999
0.13451886177062988 4370.759999999999
0.14487600326538086 4389.153499999999
0.09656453132629395 4457.306
0.1391897201538086 4476.911
0.29976511001586914 4551.579999999999
0.21927976608276367 4633.433999999999
0.20402121543884277 4731.247

- reduce를 굳이 할 필요 없을 수도 있음

# 주문 쌍 한 번씩만 확인
0.3383209705352783 4340.945499999999
0.1157827377319336 4345.277499999999
0.0524137020111084 4355.6775
0.08232855796813965 4361.633999999999
0.042936086654663086 4383.185999999999
0.03699994087219238 4410.124499999999
0.1790924072265625 4531.972999999999
0.14152073860168457 4552.808999999999
0.1615896224975586 4640.377

### 가중치 탐색을 30초 동안 하는 경우 데이터셋별 가중치 조합 몇 개를 탐색하는지 확인하기

In [52]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

for problem_file in problem_files:
    ## ------------------- 기본 변수 할당 ---------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    timelimit = 60

    ## --------------------- 커스텀 코드 시작 ---------------------------

    processed_weight_comb_c, min_init_cost = algorithm(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)

    print(problem_file, K, processed_weight_comb_c, min_init_cost)

C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K50_1.json 50 5718 3592.296
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K50_2.json 50 5900 3868.9559999999997
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K100_1.json 100 1752 3495.7119999999995
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K100_2.json 100 1823 3607.292999999999
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K200_1.json 200 426 3356.434
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K200_2.json 200 509 3313.6815
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_1.json 100 1247 3748.784000000001
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_2.json 100 925 3392.1160000000004
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_3.json 200 376 3110.1230000000005
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_4.json 200 259 2693.187999999999
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_5.json 300 123 2335.7363

### 솔루션 가능성 확인

In [3]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

for problem_file in problem_files[:2]:
    ## ------------------- 기본 변수 할당 ---------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    timelimit = 60

    ## --------------------- 커스텀 코드 시작 ---------------------------

    processed_weight_comb_c, min_init_cost, all_bundles = algorithm(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit) # all_bundles는 임시로 반환할 수 있게 추가함.

    ## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

    solution = [
            # rider type, shop_seq, dlv_seq
            [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
            for bundle in all_bundles
    ]

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    print(checked_solution['avg_cost'])

3592.296
The number of used riders of type BIKE exceeds the given available limit!


KeyError: 'avg_cost'